In [1]:
# Imports and useful definitions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
from IPython.display import display

%matplotlib inline
    
random.seed(156)

In [2]:
# Check sizes of data files
for file in os.listdir("raw_data"):
    display(file + " - " + str(round(os.path.getsize("raw_data/" + file) / 1000000, 2)) + "MB")

'cliente_tabla.csv - 21.25MB'

'producto_tabla.csv - 0.11MB'

'sample_submission.csv - 68.88MB'

'test.csv - 251.11MB'

'town_state.csv - 0.03MB'

'train.csv - 3199.36MB'

In [3]:
# Store data in DataFrames
df_clients = pd.read_csv("raw_data/cliente_tabla.csv")
df_products = pd.read_csv("raw_data/producto_tabla.csv")
df_town_state = pd.read_csv("raw_data/town_state.csv")
df_test = pd.read_csv("raw_data/test.csv")
df_train = pd.read_csv("raw_data/train.csv",
                       dtype = {'Semana' : 'int8', 'Agencia_ID' :'int32', 'Canal_ID' : 'int32', 'Ruta_SAK' : 'int32',
                                'Cliente-ID' : 'int32', 'Producto_ID':'int32', 'Venta_hoy':'float32', 'Venta_uni_hoy': 'int32', 
                                'Dev_uni_proxima':'int32', 'Dev_proxima':'float32', 'Demanda_uni_equil':'int32'})

In [4]:
# First we take a look at the clients table
display("Dimensions of Clients table : ")
display(df_clients.shape)
display("DF head : ")
display(df_clients.head())

'Dimensions of Clients table : '

(935362, 2)

'DF head : '

,Cliente_ID,NombreCliente
0,0,SIN NOMBRE
1,1,OXXO XINANTECATL
2,2,SIN NOMBRE
3,3,EL MORENO
4,4,SDN SER DE ALIM CUERPO SA CIA DE INT


In [5]:
# Rename columns in english for easier further use
df_clients.columns = ["client_id", "client_name"]
display(df_clients.head())

,client_id,client_name
0,0,SIN NOMBRE
1,1,OXXO XINANTECATL
2,2,SIN NOMBRE
3,3,EL MORENO
4,4,SDN SER DE ALIM CUERPO SA CIA DE INT


In [6]:
# Look for number of unique IDs
clientIdsUnique = len(set(df_clients["client_id"]))
clientIdsTotal = df_clients.shape[0]
clientIdsDupli = clientIdsTotal - clientIdsUnique
display("There are " + str(clientIdsUnique) + " unique Client IDs for " + str(clientIdsTotal) + " total entries")
display("There are " + str(clientIdsDupli) + " duplicate Client IDs")

# TODO : handle duplicates

# Check for missing values?
display("There are " + str(np.count_nonzero(df_clients.isnull())) + " rows with missing values")

'There are 930500 unique Client IDs for 935362 total entries'

'There are 4862 duplicate Client IDs'

'There are 0 rows with missing values'

In [7]:
# Now a look at the Products table
display("Dimensions of Products table : ")
display(df_products.shape)
display("DF head : ")
display(df_products.head())

'Dimensions of Products table : '

(2592, 2)

'DF head : '

,Producto_ID,NombreProducto
0,0,NO IDENTIFICADO 0
1,9,Capuccino Moka 750g NES 9
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41
3,53,Burritos Sincro 170g CU LON 53
4,72,Div Tira Mini Doradita 4p 45g TR 72


In [8]:
# Rename columns in english for easier further use
df_products.columns = ["product_id", "product_name"]
display(df_products.head())

,product_id,product_name
0,0,NO IDENTIFICADO 0
1,9,Capuccino Moka 750g NES 9
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41
3,53,Burritos Sincro 170g CU LON 53
4,72,Div Tira Mini Doradita 4p 45g TR 72


In [9]:
# Check for missing values?
display("There are " + str(np.count_nonzero(df_products.isnull())) + " rows with missing values")

# Augment the product DF using the data from product_name
df_products["short_name"] = df_products.product_name.str.extract("^(\D*)", expand = False)
df_products["brand"] = df_products.product_name.str.extract("^.+\s(\D+) \d+$", expand = False)
w = df_products.product_name.str.extract("(\d+)(Kg|g)", expand = True)
df_products["weight"] = w[0].astype("float")*w[1].map({"Kg":1000, "g":1})
df_products["pieces"] =  df_products.product_name.str.extract("(\d+)p ", expand = False).astype("float")
display(df_products.head())

# Check for missing values?
display("There are now " + str(np.count_nonzero(df_products.isnull())) + " rows with missing values")

'There are 0 rows with missing values'

,product_id,product_name,short_name,brand,weight,pieces
0,0,NO IDENTIFICADO 0,NO IDENTIFICADO,IDENTIFICADO,NaN,NaN
1,9,Capuccino Moka 750g NES 9,Capuccino Moka,NES,750.0,NaN
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41,Bimbollos Ext sAjonjoli,BIM,480.0,6.0
3,53,Burritos Sincro 170g CU LON 53,Burritos Sincro,LON,170.0,NaN
4,72,Div Tira Mini Doradita 4p 45g TR 72,Div Tira Mini Doradita,TR,45.0,4.0


'There are now 1585 rows with missing values'

In [10]:
# Look for number of unique IDs
productIdsUnique = len(set(df_products["product_id"]))
productIdsTotal = df_products.shape[0]
productIdsDupli = productIdsTotal - productIdsUnique
display("There are " + str(productIdsUnique) + " unique product IDs for " + str(productIdsTotal) + " total entries")
display("There are " + str(productIdsDupli) + " duplicate product IDs")

'There are 2592 unique product IDs for 2592 total entries'

'There are 0 duplicate product IDs'

In [11]:
# Now a look at the Town-State table
display("Dimensions of Town-State table : ")
display(df_town_state.shape)
display("DF head : ")
display(df_town_state.head())

'Dimensions of Town-State table : '

(790, 3)

'DF head : '

,Agencia_ID,Town,State
0,1110,2008 AG. LAGO FILT,"MÉXICO, D.F."
1,1111,2002 AG. AZCAPOTZALCO,"MÉXICO, D.F."
2,1112,2004 AG. CUAUTITLAN,ESTADO DE MÉXICO
3,1113,2008 AG. LAGO FILT,"MÉXICO, D.F."
4,1114,2029 AG.IZTAPALAPA 2,"MÉXICO, D.F."


In [12]:
# Rename columns in english for easier further use
df_town_state.columns = ["depot_id", "city", "state"]
display(df_town_state.head())

,depot_id,city,state
0,1110,2008 AG. LAGO FILT,"MÉXICO, D.F."
1,1111,2002 AG. AZCAPOTZALCO,"MÉXICO, D.F."
2,1112,2004 AG. CUAUTITLAN,ESTADO DE MÉXICO
3,1113,2008 AG. LAGO FILT,"MÉXICO, D.F."
4,1114,2029 AG.IZTAPALAPA 2,"MÉXICO, D.F."


In [13]:
# Look for number of unique IDs
depotIdsUnique = len(set(df_town_state["depot_id"]))
depotIdsTotal = df_town_state.shape[0]
depotIdsDupli = depotIdsTotal - depotIdsUnique
display("There are " + str(depotIdsUnique) + " unique depot IDs for " + str(depotIdsTotal) + " total entries")
display("There are " + str(depotIdsDupli) + " duplicate depot IDs")

# Check for missing values?
display("There are " + str(np.count_nonzero(df_town_state.isnull())) + " rows with missing values")

'There are 790 unique depot IDs for 790 total entries'

'There are 0 duplicate depot IDs'

'There are 0 rows with missing values'

In [ ]:
# TODO : What can we do with this DF?
# We could group sells by depot_id to get a general idea of the size of a depot, and use that for products on which we don't 
# have enough training data.
# I don't think there is any use for city and state.

In [14]:
# Now a look at the Train table
display("Dimensions of Train table : ")
display(df_train.shape)
display("DF head : ")
display(df_train.head())

'Dimensions of Train table : '

(74180464, 11)

'DF head : '

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4
2,3,1110,7,3301,15766,1238,4,39.320000,0,0.0,4
3,3,1110,7,3301,15766,1240,4,33.520000,0,0.0,4
4,3,1110,7,3301,15766,1242,3,22.920000,0,0.0,3


In [15]:
# Rename columns in english for easier further use
df_train.columns = ["week", "depot_id", "channel_id", "route_id", "client_id", "product_id", "sales_nb", "sales_pesos", 
                    "returns_nb", "returns_pesos", "adj_demand"]
display(df_train.head())

,week,depot_id,channel_id,route_id,client_id,product_id,sales_nb,sales_pesos,returns_nb,returns_pesos,adj_demand
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4
2,3,1110,7,3301,15766,1238,4,39.320000,0,0.0,4
3,3,1110,7,3301,15766,1240,4,33.520000,0,0.0,4
4,3,1110,7,3301,15766,1242,3,22.920000,0,0.0,3


In [16]:
# Check for missing values?
display("There are " + str(np.count_nonzero(df_train.isnull())) + " rows with missing values")

'There are 0 rows with missing values'

In [17]:
# TODO : find ou the point of channel_id and route_id

In [18]:
# Now a look at the Test table
display("Dimensions of Test table : ")
display(df_test.shape)
display("DF head : ")
display(df_test.head())

'Dimensions of Test table : '

(6999251, 7)

'DF head : '

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238
2,2,10,2045,1,2831,4549769,32940
3,3,11,1227,1,4448,4717855,43066
4,4,11,1219,1,1130,966351,1277


In [19]:
# Rename columns in english for easier further use
df_test.columns = ["id", "week", "depot_id", "channel_id", "route_id", "client_id", "product_id"]
display(df_test.head())

,id,week,depot_id,channel_id,route_id,client_id,product_id
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238
2,2,10,2045,1,2831,4549769,32940
3,3,11,1227,1,4448,4717855,43066
4,4,11,1219,1,1130,966351,1277


In [20]:
# Now a detailed look at the Train table
df_train.describe()

,week,depot_id,channel_id,route_id,client_id,product_id,sales_nb,sales_pesos,returns_nb,returns_pesos,adj_demand
count,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07
mean,5.950021e+00,2.536509e+03,1.383181e+00,2.114855e+03,1.802119e+06,2.084081e+04,7.310163e+00,4.926807e+01,1.302577e-01,1.241306e+00,7.224564e+00
std,2.013175e+00,4.075124e+03,1.463266e+00,1.487744e+03,2.349577e+06,1.866392e+04,2.196734e+01,3.357969e+02,2.932320e+01,3.907285e+01,2.177119e+01
min,3.000000e+00,1.110000e+03,1.000000e+00,1.000000e+00,2.600000e+01,4.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.000000e+00,1.312000e+03,1.000000e+00,1.161000e+03,3.567670e+05,1.242000e+03,2.000000e+00,1.676000e+01,0.000000e+00,0.000000e+00,2.000000e+00
50%,6.000000e+00,1.613000e+03,1.000000e+00,1.286000e+03,1.193385e+06,3.054900e+04,3.000000e+00,3.000000e+01,0.000000e+00,0.000000e+00,3.000000e+00
75%,8.000000e+00,2.036000e+03,1.000000e+00,2.802000e+03,2.371091e+06,3.742600e+04,7.000000e+00,5.610000e+01,0.000000e+00,0.000000e+00,6.000000e+00
max,9.000000e+00,2.575900e+04,1.100000e+01,9.991000e+03,2.015152e+09,4.999700e+04,7.200000e+03,6.473600e+05,2.500000e+05,1.307600e+05,5.000000e+03


In [27]:
df_train[(df_train.product_id == 1212) & (df_train.client_id == 15766)] 

,week,depot_id,channel_id,route_id,client_id,product_id,sales_nb,sales_pesos,returns_nb,returns_pesos,adj_demand
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
11165209,4,1110,7,3301,15766,1212,4,33.520000,0,0.0,4
22174800,5,1110,7,3301,15766,1212,5,41.900002,0,0.0,5
53364883,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,4
63771751,9,1110,7,3301,15766,1212,1,8.380000,0,0.0,1


In [ ]:
# DataFrame is so big, it takes too much time to plot or do detailed analysis on it.
# let's sample 1% of it for the time being.
sample_size = int(df_train.shape[0] * 0.01)
display("Sample size : " + str(sample_size))
sample_rows = np.random.choice(df_train.index.values, sample_size)
df_train_small = df_train.ix[sample_rows]
display("DF head : ")
display(df_train_small.head())

In [ ]:
# Plot the data
target = df_train_small["adj_demand"].tolist()
plt.hist(target, bins = 50, color = "red")
plt.title("Distribution of target variable")
plt.xlabel("Adjusted demand")
plt.ylabel("Count")
plt.show()

In [ ]:
# Distribution of target variable (adj_demand) seems to have some outliers at the top end, which render the plot useless.
# 75th percentile is "6", but max is "5000".
display("Target values under 500 : " + str(round(len(df_train_small.loc[df_train_small["adj_demand"] <= 500]) / 7418.04, 2)) + "%")
display("Target values under 100 : " + str(round(len(df_train_small.loc[df_train_small["adj_demand"] <= 100]) / 7418.04, 2)) + "%")
display("Target values under 50 : " + str(round(len(df_train_small.loc[df_train_small["adj_demand"] <= 50]) / 7418.04, 2)) + "%")

In [ ]:
# Let's plot it again, zooming on values under 100, thus keeping more than 99.5% of all rows.
target = df_train_small["adj_demand"].tolist()
plt.hist(target, bins = 100, color = "red", range = (0, 100))
plt.title("Distribution of target variable under 100")
plt.xlabel("Adjusted demand")
plt.ylabel("Count")
plt.show()

In [ ]:
# Looks like a Poisson distribution?
# A real uptick around the value 40, to investigate?
display("The most common values are, in order : 2 then 1 then 3")

In [ ]:
# Order by date then depot ID
df_train_small = df_train_small.sort_values(by = ["week", "client_id", "product_id"], ascending = True)
display("DF head : ")
display(df_train_small.head())
display("DF tail : ")
display(df_train_small.tail())

In [ ]:
# Let's see if our random sampling caught approximately equal number of rows from all weeks
plt.hist(df_train_small["week"].tolist(), bins = 7, color = "red")
plt.title("Distribution of weeks in sampled data")
plt.xlabel("week")
plt.ylabel("Count")
plt.show()
display("Seems we have enough rows for each week")

In [ ]:
# Let's look for time patterns in the target value
# Only look at values < 25 to have a usable plot
df_train_small_time = df_train_small.loc[df_train_small["adj_demand"] < 25]
plt.hist2d(df_train_small_time["week"].tolist(), df_train_small_time["adj_demand"].tolist(), bins = [7, 25])
plt.title("Distribution of target value over time")
plt.xlabel("week")
plt.ylabel("adj_demand")
plt.show()
display("There does seem to be a time pattern, adj_demand seems to decline from week 3 to week 6 for all levels.")
display("The values then tend to mostly stay at their week 6 (and even more week 7) value until week 9.")
display("It may make sense to only look at week 7 to week 9 data if one wants to sample down.")